In [1]:
########################################################################
# II. PREDICT AVERAGE SESSION LENGTH
########################################################################
import pandas as pd
import pickle
from sklearn.linear_model import *
from sklearn.ensemble import *
from sklearn.model_selection import *



In [6]:
# method to find IP address class
def find_class(ip):
    a=int(ip.split('.')[0])
    
  
    # Class A 
    if (a >=1 and a <= 126) :
        return 1 
  
    # Class B 
    elif (a >= 128 and a <= 191): 
        return 2
  
    # Class C 
    elif (a >= 192 and a <= 223) :
        return 3 
  
    # Class D 
    elif (a >= 224 and a <= 239) :
        return 4 
    # Class E
    else:
        return 5 


In [ ]:
# Read output of URL hits from DE section

hits = pd.read_csv('/Users/bjayaram/Downloads/WeblogChallenge-master/data/output/url_hits/part-00000-d78ec039-454d-42c2-908a-bf9fd79a2248-c000.csv',low_memory=False)
print hits.columns

data = hits[['session_id','ip','unique_hits']].groupby(['session_id','ip'],as_index=False).sum()


In [11]:

# Split IP address into 4 features
X = data['ip'].str.split('.', expand=True)
for i in range(4):
    c = chr(ord('a') + i)
    X[c] = map(int, X[i])


In [12]:
# Add session id
X['session_id'] = data['session_id']

# find class (i.e A, B, C, D) for each IP address. Method find_class() defined in parse_ip.py
X['class'] = map(find_class, data['ip'])


In [12]:
#  X contains IP class, session id, 4 components of IP address
X = X[['class', 'session_id', 'a', 'b', 'c', 'd']].values

# Y contains number of unique URL hits
Y = data['unique_hits'].values
print "Data extracted", X.shape, Y.shape


Data extracted (276932, 6) (276932,)


In [13]:
alpha_list = range(10,10000,10)
alphas = map(lambda x: 10.0/(1+x), alpha_list)
results={}


# Try the following regressors: 
# 1.Linear Regression, 
# 2.GradientBoostingRegressor, 
# 3.Ridge, 
# 4.Lasso, 
# 5.RandomForest
model1 = GradientBoostingRegressor(n_estimators=1000)
model2 = RidgeCV(normalize=True, alphas = alphas)
model3 = LassoCV(normalize=True, alphas = alphas, max_iter=1000)
model4 = RandomForestRegressor(n_estimators=1000)
model5 = LinearRegression(normalize=True)


In [ ]:
# Try the following regressors: Linear Regression, GradientBoostingRegressor, Ridge, Lasso, RandomForest
model1 = GradientBoostingRegressor(n_estimators=100)
model2 = RidgeCV(normalize=True, alphas = alphas)
model3 = LassoCV(normalize=True, alphas = alphas, max_iter=1000)
model4 = RandomForestRegressor(n_estimators=1000)
model5 = LinearRegression(normalize=True)


# Run 10-fold cross validation on each of the models to evaluate the performance
print "Evaluating Gradient Boosting regressor"
results1 = cross_val_score(model1, X, Y, cv=10, n_jobs=-1,scoring='neg_mean_squared_error')

print "Evaluating Ridge regressor"
results2 = cross_val_score(model2, X, Y, cv=10, n_jobs=-1,scoring='neg_mean_squared_error')

print "Evaluating Lasso regressor"
results3 = cross_val_score(model3, X, Y, cv=10, n_jobs=-1,scoring='neg_mean_squared_error')

print "Evaluating Random Forest regressor"
results4 = cross_val_score(model4, X, Y, cv=10, n_jobs=-1,scoring='neg_mean_squared_error')

print "Evaluating Linear regressor"
results5 = cross_val_score(model5, X, Y, cv=10, n_jobs=-1,scoring='neg_mean_squared_error')


results['Gradient Boosting Regression']=results1.mean()
results['Ridge Regresison']=results2.mean()
results['Lasso Regression']=results3.mean()
results['RandomForestRegressor']=results4.mean()
results['Linear Regression']=results5.mean()

"""

print results
# -ve MSE (in minutes) values are as follows:
# 'Ridge Regresison': -30.641001729376807,
# 'Lasso Regression': -30.608556949805347,
# 'Gradient Boosting Regression': -30.054393398542505,
# 'RandomForestRegressor': -32.60778837997033}
"""


In [ ]:
# Further iterate with Gradient Boosting Regressor


lr = map(lambda x:10 ** -x, range(1,4))

model=GridSearchCV(GradientBoostingRegressor(),n_jobs=-1, cv=10,scoring='neg_mean_squared_error',verbose=1,
                   param_grid={'learning_rate':lr,'n_estimators':[100,1000,10000]})
model.fit(X,Y)




print "BEST SCORE", model.best_score_, model.scoring
print "BEST ESTIMATOR", model.best_estimator_
pickle.dump(model,open('/Users/bjayaram/Downloads/WeblogChallenge-master/WebLogsPrediction/output/intervals_model.p','w'))
# BEST SCORE: 29.9875736945 hits neg_mean_squared_error
# BEST ESTIMATOR:
# GridSearchCV(cv=10, error_score='raise',
#        estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
#              learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
#              max_leaf_nodes=None, min_impurity_decrease=0.0,
#              min_impurity_split=None, min_samples_leaf=1,
#              min_samples_split=2, min_weight_fraction_leaf=0.0,
#              n_estimators=100, presort='auto', random_state=None,
#              subsample=1.0, verbose=0, warm_start=False),
#        fit_params=None, iid=True, n_jobs=-1,
#        param_grid={'n_estimators': [100], 'learning_rate': [0.1, 0.01, 0.001]},
#        pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
#        scoring='neg_mean_squared_error', verbose=1)





